In [1]:
include("symmetrySpectral.jl")
type GraphNodeType
    Tran::Array{Float64}
    SymType::AbstractString
end

In [2]:
type GraphEdge
    v1::Tuple{Int32, Int32}
    v2::Tuple{Int32, Int32}
    dist::Float64
end

In [3]:
conn=[]
v = GraphNodeType(eye(4), "C4v")
v.Tran[1,4] = 1
push!(conn,v)
v = GraphNodeType(eye(4), "C4v")
v.Tran[3,4] = 1
push!(conn,v)
v = GraphNodeType(eye(4), "C4v")
v.Tran[1,4] = -1
push!(conn,v)
v = GraphNodeType(eye(4), "C4v")
v.Tran[3,4] = -1
push!(conn,v)

4-element Array{Any,1}:
 GraphNodeType(4x4 Array{Float64,2}:
 1.0  0.0  0.0  1.0
 0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  0.0  1.0,"C4v")    
 GraphNodeType(4x4 Array{Float64,2}:
 1.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  0.0  1.0  1.0
 0.0  0.0  0.0  1.0,"C4v")    
 GraphNodeType(4x4 Array{Float64,2}:
 1.0  0.0  0.0  -1.0
 0.0  1.0  0.0   0.0
 0.0  0.0  1.0   0.0
 0.0  0.0  0.0   1.0,"C4v")
 GraphNodeType(4x4 Array{Float64,2}:
 1.0  0.0  0.0   0.0
 0.0  1.0  0.0   0.0
 0.0  0.0  1.0  -1.0
 0.0  0.0  0.0   1.0,"C4v")

In [4]:
SymGroup = []
matrix = axis2matrix([0,1,0],4)
reflect = axis2matrix([0,0,1],-1)
temp = matrix
for i = 1:4
    push!(SymGroup, matrix)
    matrix = matrix * temp
    push!(SymGroup, reflect)
    reflect = reflect * temp
end
for i = 1:size(SymGroup,1)
    temp = SymGroup[i]
    SymGroup[i] = eye(4)
    SymGroup[i][1:3,1:3]=temp
end

In [5]:
Transformations = Array{Array{Float64,2}}(12,8)
count = 0
for i = 1:4
    for j = 1:4
        if i!= j 
            count += 1
            for k = 1:size(SymGroup,1)
                Transformations[count,k]=conn[j].Tran*SymGroup[k]*inv(conn[i].Tran)
            end
        end
    end
end     

In [6]:
EdgeSet = []
dist = zeros(8,8)
for i = 1:size(Transformations,1)
    for j = i+1 : size(Transformations,1)
        for gi = 1:size(Transformations,2)
            for gj = 1:size(Transformations,2)
                dist[gi,gj] = norm(Transformations[i,gi][:]-Transformations[j,gj][:])
            end
        end
        idx = find(dist.<0.3)
        for k in idx
            gj = mod(k-1, size(Transformations,2)) + 1
            gi = div(k-1, size(Transformations,2)) + 1
            push!(EdgeSet, GraphEdge((i,gi),(j,gj), dist[k]))
        end
    end
end
EdgeSet

52-element Array{Any,1}:
 GraphEdge((1,3),(4,3),6.280369834735101e-16)  
 GraphEdge((1,4),(4,4),3.1401849173675503e-16) 
 GraphEdge((1,2),(5,2),0.0)                    
 GraphEdge((1,5),(5,5),9.42055475210265e-16)   
 GraphEdge((1,4),(9,4),4.440892098500626e-16)  
 GraphEdge((1,5),(9,5),1.3322676295501878e-15) 
 GraphEdge((1,5),(10,5),9.42055475210265e-16)  
 GraphEdge((1,6),(10,6),6.280369834735101e-16) 
 GraphEdge((1,4),(12,4),3.1401849173675503e-16)
 GraphEdge((1,7),(12,7),1.2560739669470201e-15)
 GraphEdge((2,3),(6,3),6.280369834735101e-16)  
 GraphEdge((2,4),(6,4),3.1401849173675503e-16) 
 GraphEdge((2,3),(7,3),8.881784197001252e-16)  
 ⋮                                             
 GraphEdge((6,2),(11,2),0.0)                   
 GraphEdge((6,3),(11,3),8.881784197001252e-16) 
 GraphEdge((7,3),(11,3),6.280369834735101e-16) 
 GraphEdge((7,4),(11,4),3.1401849173675503e-16)
 GraphEdge((8,7),(10,7),1.2560739669470201e-15)
 GraphEdge((8,8),(10,8),9.42055475210265e-16)  
 GraphEdge((8,1

In [15]:
function FindRoot!(C, v)
    if C[v] != v
        C[v] = FindRoot!(C, C[v])
    end
    C[v]
end

function GetConnectedComponents(EdgeSet)
    parent = Dict{Tuple{Int32, Int32}, Tuple{Int32, Int32}}()
    for edge in EdgeSet
        if !haskey(parent, edge.v1)
            parent[edge.v1]=edge.v1
        end
        if !haskey(parent, edge.v2)
            parent[edge.v2]=edge.v2
        end
        root1 = FindRoot!(parent, edge.v1)
        root2 = FindRoot!(parent, edge.v2)
        parent[root2] = root1
    end
    for vert in keys(parent)
        parent[vert] = FindRoot!(parent, vert)
    end

    components = Dict{Tuple{Int32, Int32}, Array{Tuple{Int32, Int32}}}()
    for k in keys(parent)
        if !haskey(components, parent[k])
            components[parent[k]] = []
        end
        push!(components[parent[k]], k)
    end    
    components
end

GetConnectedComponents (generic function with 1 method)

In [18]:
components = GetConnectedComponents(EdgeSet)
showall(components)

Dict{Tuple{Int32,Int32},Array{Tuple{Int32,Int32},N}}((1,6)=>[(10,6),(1,6)],(3,6)=>[(3,6),(4,6)],(2,3)=>[(2,3),(6,3),(11,3),(7,3)],(2,6)=>[(2,6),(7,6)],(5,6)=>[(9,6),(5,6)],(8,6)=>[(12,6),(8,6)],(1,7)=>[(12,7),(1,7)],(3,1)=>[(3,1),(4,1),(8,1),(12,1)],(4,7)=>[(4,7),(9,7)],(1,4)=>[(12,4),(4,4),(9,4),(1,4)],(6,8)=>[(7,8),(6,8)],(7,4)=>[(11,4),(7,4)],(3,3)=>[(3,3),(10,3)],(2,8)=>[(11,8),(2,8)],(6,2)=>[(6,2),(11,2)],(1,3)=>[(1,3),(4,3)],(8,7)=>[(8,7),(10,7)],(3,2)=>[(3,2),(12,2)],(2,4)=>[(2,4),(6,4)],(9,3)=>[(9,3),(12,3)],(1,2)=>[(1,2),(5,2)],(9,2)=>[(9,2),(10,2)],(3,7)=>[(5,7),(3,7)],(5,3)=>[(8,3),(5,3)],(1,5)=>[(10,5),(5,5),(1,5),(9,5)],(4,2)=>[(8,2),(4,2)],(3,8)=>[(5,8),(10,8),(8,8),(3,8)])